# Predicting Residential EV Charging Loads using Neural Networks

### a model like this could be useful in predicting things like energy costs when developing EV charging infrastructure.

Data : https://data.mendeley.com/datasets/jbks2rcwyj/1

### Data Features
- session_ID - the unique id for each EV charging session
- Garage_ID - the unique id for the garage of the apartment
- User_ID - the unique id for each user
- User_private - 1.0 indicates private charge point spaces and 0.0 indicates shared charge point spaces
- Shared_ID - the unique id if shared charge point spaces are used
- Start_plugin - the plug-in date and time in the format (day.month.year hour:minute)
- Start_plugin_hour - the plug-in date and time rounded to the start of the hour
- End_plugout - the plug-out date and time in the format (day.month.year hour:minute)
- End_plugout_hour - the start of the hour of the End_plugout hour
- El_kWh - the charged energy in kWh (charging loads)
- Duration_hours - the duration of the EV connection time per session
- Plugin_category - the plug-in time categorized by early/late night, morning, afternoon, and evening
- Duration_category - the plug-in duration categorized by 3 hour groups
- month_plugin_{month} - the month of the plug-in session
- weekdays_plugin_{day} - the day of the week of the plug-in session


Target to predict

1. the actual charging load in kilowatt hours for a charging session

In [1]:
# Setup - import basic data libraries
import numpy as np
import pandas as pd

## Load, Inspect, and Merge Datasets

### Step 1

The file `'datasets/EV charging reports.csv'` contains electric vehicle (EV) charging data. These come from various residential apartment buildings in Norway. The data includes specific user and garage information, plug-in and plug-out times, charging loads, and the dates of the charging sessions.

Import this CSV file to a pandas DataFrame named `ev_charging_reports`.

Use the `.head()` method to preview the first five rows.

In [2]:
ev_charging_reports = pd.read_csv("datasets/EV charging reports.csv")
ev_charging_reports.head()

,session_ID,Garage_ID,User_ID,User_private,Shared_ID,Start_plugin,Start_plugin_hour,End_plugout,End_plugout_hour,El_kWh,...,month_plugin_Nov,month_plugin_Oct,month_plugin_Sep,weekdays_plugin_Friday,weekdays_plugin_Monday,weekdays_plugin_Saturday,weekdays_plugin_Sunday,weekdays_plugin_Thursday,weekdays_plugin_Tuesday,weekdays_plugin_Wednesday
0,1,AdO3,AdO3-4,1.0,NaN,21.12.2018 10:20,21.12.2018 10:00,21.12.2018 10:23,10.0,"0,3",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,AdO3,AdO3-4,1.0,NaN,21.12.2018 10:24,21.12.2018 10:00,21.12.2018 10:32,10.0,"0,87",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,AdO3,AdO3-4,1.0,NaN,21.12.2018 11:33,21.12.2018 11:00,21.12.2018 19:46,19.0,"29,87",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,AdO3,AdO3-2,1.0,NaN,22.12.2018 16:15,22.12.2018 16:00,23.12.2018 16:40,16.0,"15,56",...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,AdO3,AdO3-2,1.0,NaN,24.12.2018 22:03,24.12.2018 22:00,24.12.2018 23:02,23.0,"3,62",...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<details><summary style="display:list-item; font-size:16px; color:blue;">What is the structure of the dataset?</summary>

- **session_ID** - the unique id for each EV charging session
- **Garage_ID** - the unique id for the garage of the apartment
- **User_ID** - the unique id for each user
- **User_private** - 1.0 indicates private charge point spaces and 0.0 indicates shared charge point spaces
- **Shared_ID** - the unique id if shared charge point spaces are used
- **Start_plugin** - the plug-in date and time in the format (day.month.year hour:minute)
- **Start_plugin_hour** - the plug-in date and time rounded to the start of the hour
- **End_plugout** - the plug-out date and time in the format (day.month.year hour:minute)
- **End_plugout_hour** - the start of the hour of the `End_plugout` hour
- **El_kWh** - the charged energy in kWh (charging loads)
- **Duration_hours** - the duration of the EV connection time per session
- **Plugin_category** - the plug-in time categorized by early/late night, morning, afternoon, and evening
- **Duration_category** - the plug-in duration categorized by 3 hour groups
- **month_plugin_{month}** - the month of the plug-in session
- **weekdays_plugin_{day}** - the day of the week of the plug-in session

### Step 2

Import the file `'datasets/Local traffic distribution.csv'` to a pandas DataFrame named `traffic_reports`. This dataset contains the hourly local traffic density counts at 5 nearby traffic locations. 

Preview the first five rows.

In [3]:
traffic_reports = pd.read_csv('datasets/Local traffic distribution.csv')
traffic_reports.head()

,Date_from,Date_to,Kroppan_bru_traffic,Moholtlia_traffic,Selsbakk_traffic,Moholt_rampe_2_traffic,Jonsvannsveien_vest_steinanvegen_traffic
0,01.12.2018 00:00,01.12.2018 01:00,639,0,0,4,144
1,01.12.2018 01:00,01.12.2018 02:00,487,153,115,21,83
2,01.12.2018 02:00,01.12.2018 03:00,408,85,75,10,69
3,01.12.2018 03:00,01.12.2018 04:00,282,89,56,8,39
4,01.12.2018 04:00,01.12.2018 05:00,165,64,34,3,25


<details><summary style="display:list-item; font-size:16px; color:blue;">What is the structure of the dataset?</summary>

- **Date_from** - the starting time in the format (day.month.year hour:minute)
- **Date_to** - the ending time in the format (day.month.year hour:minute)
- **Location 1 to 5** - contains the number of vehicles each hour at a specified traffic location.


### Step 3

We'd like to use the traffic data to help our model. The same charging location may charge at different rates depending on the number of cars being charged, so this traffic data might help the model out.

Merge the `ev_charging_reports` and `traffic_reports` datasets together into a Dataframe named `ev_charging_traffic` using the columns:

- `Start_plugin_hour` in `ev_charging_reports`
- `Date_from` in `traffic_reports`

In [4]:
ev_charging_traffic = pd.merge(ev_charging_reports, traffic_reports, left_on='Start_plugin_hour', right_on='Date_from')
ev_charging_traffic.head()

,session_ID,Garage_ID,User_ID,User_private,Shared_ID,Start_plugin,Start_plugin_hour,End_plugout,End_plugout_hour,El_kWh,...,weekdays_plugin_Thursday,weekdays_plugin_Tuesday,weekdays_plugin_Wednesday,Date_from,Date_to,Kroppan_bru_traffic,Moholtlia_traffic,Selsbakk_traffic,Moholt_rampe_2_traffic,Jonsvannsveien_vest_steinanvegen_traffic
0,1,AdO3,AdO3-4,1.0,NaN,21.12.2018 10:20,21.12.2018 10:00,21.12.2018 10:23,10.0,"0,3",...,0.0,0.0,0.0,21.12.2018 10:00,21.12.2018 11:00,3244,1632,545,194,622
1,2,AdO3,AdO3-4,1.0,NaN,21.12.2018 10:24,21.12.2018 10:00,21.12.2018 10:32,10.0,"0,87",...,0.0,0.0,0.0,21.12.2018 10:00,21.12.2018 11:00,3244,1632,545,194,622
2,3,AdO3,AdO3-4,1.0,NaN,21.12.2018 11:33,21.12.2018 11:00,21.12.2018 19:46,19.0,"29,87",...,0.0,0.0,0.0,21.12.2018 11:00,21.12.2018 12:00,3605,1691,605,230,771
3,4,AdO3,AdO3-2,1.0,NaN,22.12.2018 16:15,22.12.2018 16:00,23.12.2018 16:40,16.0,"15,56",...,0.0,0.0,0.0,22.12.2018 16:00,22.12.2018 17:00,3052,1484,453,224,694
4,5,AdO3,AdO3-2,1.0,NaN,24.12.2018 22:03,24.12.2018 22:00,24.12.2018 23:02,23.0,"3,62",...,0.0,0.0,0.0,24.12.2018 22:00,24.12.2018 23:00,1390,693,226,83,353


### Task 4

Use `.info()` to inspect the merged dataset. Specifically, pay attention to the data types and number of missing values in each column.

In [5]:
ev_charging_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6833 entries, 0 to 6832
Data columns (total 39 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   session_ID                                6833 non-null   int64  
 1   Garage_ID                                 6833 non-null   object 
 2   User_ID                                   6833 non-null   object 
 3   User_private                              6833 non-null   float64
 4   Shared_ID                                 1399 non-null   object 
 5   Start_plugin                              6833 non-null   object 
 6   Start_plugin_hour                         6833 non-null   object 
 7   End_plugout                               6833 non-null   object 
 8   End_plugout_hour                          6833 non-null   float64
 9   El_kWh                                    6833 non-null   object 
 10  Duration_hours                      

<details><summary style="display:list-item; font-size:16px; color:blue;">What do we notice about merged dataset under inspection?</summary>

We see that there are 39 columns and 6,833 rows in our merged dataset.

Some notable things we might have to address:

- We expected columns like `El_kWh` and `Duration_hours` to be floats but they are actually object data types.

- There are many identifying columns like `session_ID` and `User_ID` that might not be useful for training.

##  Data Cleaning and Preparation

### Step 5

Let's start by reducing the size of our dataset by dropping columns that won't be used for training. These include
- ID columns
- columns with lots of missing data
- non-numeric columns (for now, since we haven't yet covered using non-numeric data in neural networks)

Drop columns you don't want to use in training from `ev_charging_traffic`.

To match our solution, drop the columns

```py
['session_ID', 'Garage_ID', 'User_ID', 
                'Shared_ID',
                'Plugin_category','Duration_category', 
                'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', 
                'Date_from', 'Date_to']
```

In [6]:
ev_charging_traffic = ev_charging_traffic.drop(['session_ID', 'Garage_ID', 'User_ID', 'Shared_ID', 'Plugin_category', 'Duration_category', 'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', 'Date_from', 'Date_to'], axis=1)


### Step 6

Earlier we saw that the `El_kWh` and `Duration_hours` columns were object data types. Upon further inspection, we see that the reason is that the data is following European notation where commas `,` are used as decimals instead of periods.

Replace `,` with `.` in these three columns.

In [7]:
ev_charging_traffic['El_kWh'] = ev_charging_traffic['El_kWh'].str.replace(',','.')
ev_charging_traffic['Duration_hours'] = ev_charging_traffic['Duration_hours'].str.replace(',','.')

### Step 7

Next, convert the data types of all the columns of `ev_charging_traffic` to floats.

In [8]:
ev_charging_traffic = ev_charging_traffic.astype(float)

## Train Test Split

Next, let's split the dataset into training and testing datasets. 

The training data will be used to train the model and the testing data will be used to evaluate the model.

### Step 8

First, create two datasets from `ev_charging_traffic`:

- `X` contains only the input numerical features
- `y` contains only the target column `El_kWh`

In [9]:
from sklearn.model_selection import train_test_split
X = ev_charging_traffic.drop('El_kWh', axis=1)
y = ev_charging_traffic['El_kWh']

### Step 9

Use `sklearn` to split `X` and `y` into training and testing datasets. The training set should use 80% of the data. Set the `random_state` parameter to `2`.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
    train_size=0.80,
    test_size=0.20,
    random_state=2)

## Linear Regression Baseline

This section is optional, but useful. The idea is to compare our neural network to a basic linear regression. After all, if a basic linear regression works just as well, there's no need for the neural network!

If you haven't done linear regression with scikit-learn before, feel free to use [our solution code](./solutions.html) or to skip ahead.

### Step 10

Use Scikit-learn to train a Linear Regression model using the training data to predict EV charging loads.

The linear regression will be used as a baseline to compare against the neural network we will train later.

In [11]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)


LinearRegression()

### Step 11

Evaluate the linear regression baseline by calculating the MSE on the testing data. Use `mean_squared_error` from `sklearn.metrics`.

Save the testing MSE to the variable `test_mse` and print it out.

In [13]:
from sklearn.metrics import mean_squared_error

predictions = linear_model.predict(X_test)

test_mse = mean_squared_error(y_test, predictions)
print(f"The mean squared error is: {test_mse}")

The mean squared error is: 131.4188163356643


Looks like our mean squared error is around `131.4` (if you used different columns in your model than we did, you might have a different value). Remember, this is squared error. If we take the square root, we have about `11.5`. One way of interpreting this is to say that the linear regression, on average, is off by `11.5 kWh`.

## Train a Neural Network Using PyTorch

Let's now create a neural network using PyTorch to predict EV charging loads.

### Task 12

First, we'll need to import the PyTorch library and modules.

Import the PyTorch library `torch`.

From `torch`, import `nn` to access built-in code for constructing networks and defining loss functions.

From `torch`, import `optim` to access built-in optimizer algorithms.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

### Step 13

Before training the neural network, convert the training and testing sets into PyTorch tensors and specify `float` as the data type for the values.

In [23]:
#Use the .view(-1, 1) method to reshape the y_train and y_test tensors for training.
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

### Step 14

Next, let's use `nn.Sequential` to create a neural network.

First, set a random seed using `torch.manual_seed(42)`.

Then, create a sequential neural network with the following architecture:

- input layer with number of nodes equal to the number of training features
- a first hidden layer with `56` nodes and a ReLU activation
- a second hidden layer with `26` nodes and a ReLU activation
- an output layer with `1` node

Save the network to the variable `model`.

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5466 entries, 839 to 2575
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   User_private                              5466 non-null   float64
 1   Duration_hours                            5466 non-null   float64
 2   month_plugin_Apr                          5466 non-null   float64
 3   month_plugin_Aug                          5466 non-null   float64
 4   month_plugin_Dec                          5466 non-null   float64
 5   month_plugin_Feb                          5466 non-null   float64
 6   month_plugin_Jan                          5466 non-null   float64
 7   month_plugin_Jul                          5466 non-null   float64
 8   month_plugin_Jun                          5466 non-null   float64
 9   month_plugin_Mar                          5466 non-null   float64
 10  month_plugin_May                       

In [20]:
torch.manual_seed(42)

# create neural network
model = nn.Sequential(
    nn.Linear(26,56),
    nn.ReLU(),
    nn.Linear(56,26),
    nn.ReLU(),
    nn.Linear(26,1)
)

### Step 15

Next, let's define the loss function and optimizer used for training:

- set the MSE loss function to the variable `loss`
- set the Adam optimizer to the variable `optimizer` with a learning rate of `0.0007`

In [21]:
optimizer = optim.Adam(model.parameters(), lr=0.0007)

# define the loss function and compute loss
loss = nn.MSELoss()

### Step 16

Create a training loop to train our neural network for 3000 epochs.

Keep track of the training loss by printing out the MSE every 500 epochs.

In [22]:
num_epochs = 3000
for epoch in range(num_epochs):
    predictions = model(X_train_tensor) 
    MSE = loss(predictions, y_train_tensor)
    MSE.backward()
    optimizer.step()
    optimizer.zero_grad()

    # keep track of the loss during training
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], MSE Loss: {MSE.item()}')

Epoch [500/3000], MSE Loss: 146.96762084960938
Epoch [1000/3000], MSE Loss: 138.64056396484375
Epoch [1500/3000], MSE Loss: 138.99111938476562
Epoch [2000/3000], MSE Loss: 137.61985778808594
Epoch [2500/3000], MSE Loss: 138.0702667236328
Epoch [3000/3000], MSE Loss: 137.82913208007812


### Step 17

Save the neural network in the `models` directory using the path `models/model.pth`.

In [24]:
# save the neural network to a specified path
torch.save(model, 'models/model.pth')

### Step 18

Evaluate the neural network on the testing set. 

Save the testing data loss to the variable `test_loss` and use `.item()` to extract and print out the loss. 

In [25]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_MSE = loss(predictions, y_test_tensor)
    
print('Test MSE is ' + str(test_MSE.item()))
print('Test Root MSE is ' + str(test_MSE.item()**(1/2)))

Test MSE is 156.2063751220703
Test Root MSE is 12.498254883065488


### Step 19

We trained this same model for 4500 epochs locally. That model is saved as `models/model4500.pth`. Load this model using PyTorch and evaluate it. How well does the longer-trained model perform?

In [26]:
loaded_model = torch.load('models/model4500.pth')
loaded_model.eval()
with torch.no_grad():
    predictions = loaded_model(X_test_tensor)
    test_MSE = loss(predictions, y_test_tensor)

## DO NOT MODIFY ##
# show output
print('Test MSE is ' + str(test_MSE.item()))
print('Test Root MSE is ' + str(test_MSE.item()**(1/2)))

Test MSE is 115.21600341796875
Test Root MSE is 10.733871781327032


Pretty cool! The increased training improved our test loss to about `115.2`, a full `12%` improvement on our linear regression baseline. So the nonlinearity introduced by the neural network actually helped us out.

That's the end of our project on predicting EV charging loads! Feel free to continue experimenting with this neural network model. 

Some things you might want to investigate further include:
- explore different ways to clean and prepare the data
- we added traffic data, but there's no guarantee that more data converts to a better model. Test out different sets of input columns.
- test out different number of nodes in the hidden layers, activation functions, and learning rates
- train on a larger number of epochs 